In [8]:
# !pip install transformers datasets

In [9]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True) # thêm force_remount=True để đảm bảo ổ đĩa được mount lại

Mounted at /content/drive


In [10]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
import torch

# Tải dữ liệu từ Hugging Face

In [11]:
dataset = load_dataset("xDAN-datasets/ChatDoctor_HealthCareMagic_112k")

# Lấy ra một phần dữ liệu

In [12]:
dataset = dataset['train'].select(range(5000))

# Kiểm tra dữ liệu

In [13]:
# print(dataset['train'][0])

# Tải tokenizer và model từ GPT-2

In [18]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Chuẩn bị dữ liệu huấn luyện

In [20]:
conversations = dataset['conversations']

def preprocess_data(conversations):
    inputs = []
    outputs = []

    for conversation in conversations:
        # Iterate through the list of dictionaries in 'conversation'
        for turn in conversation:
            # Now access the 'input' and 'output' keys in each dictionary
            input_text = turn.get('input')
            output_text = turn.get('output')

            # Only append if both 'input' and 'output' are present and not empty strings
            if input_text and output_text and input_text.strip() and output_text.strip():  # Check if strings are not empty after stripping whitespace
                inputs.append(input_text)
                outputs.append(output_text)

    # Check if inputs or outputs are empty after filtering
    if not inputs or not outputs:
        print("Warning: Empty inputs or outputs after filtering. Returning empty tensors.")
        return {}, {}  # Return empty dictionaries to avoid tokenizer error

    # Token hóa các câu hỏi và câu trả lời
    tokenized_inputs = tokenizer(inputs, padding=True, truncation=True, return_tensors="pt")
    tokenized_outputs = tokenizer(outputs, padding=True, truncation=True, return_tensors="pt")

    return tokenized_inputs, tokenized_outputs

# Tiền xử lý dữ liệu hội thoại
tokenized_inputs, tokenized_outputs = preprocess_data(conversations)


# Thiết lập các tham số cho quá trình fine-tuning

In [21]:
# Thiết lập tham số huấn luyện
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    num_train_epochs=3,  # Số lượng epoch để huấn luyện
    logging_dir="./logs",
    logging_steps=200,
    save_steps=1000,
    save_total_limit=2,
    evaluation_strategy="steps",  # Đánh giá theo từng bước
    eval_steps=500,
    load_best_model_at_end=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


# Tạo một Trainer để fine-tune mô hình

In [23]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

trainer.train()

NameError: name 'tokenized_datasets' is not defined

# Lưu mô hình đã fine-tune

In [ ]:
model.save_pretrained("./fine_tuned_chatdoctor")
tokenizer.save_pretrained("./fine_tuned_chatdoctor")